# Practical Week 5: Evolutionary Algorithms

In this practical you will implement a simple Genetic Algorithm and apply it on examples. This work is to build your understanding of evolutionary search algorithms and your ability to apply them to examples. This work is not assessed, and you do not need to submit it. Please ask questions if you are facing difficulties with any of the content in this practical.

Genetic algorithms (or GA) are a form of evolutionary algorithm that are inspired by natural evolution. GAs are particularly useful in optimization and search problems with large state spaces that cannot be solved mathematically.

Given a problem, GAs make use of a population of individuals that represent candidate solutions. Each such individual is represented by a chromosome that encodes the candidate solution in the form of genes, such that the candidate solution can be reconstructed from the corresponding chromosome. GAs work in iterations, where a new population is generated from the previous population in each iteration. This process is guided by a fitness function, which captures the quality of a candidate solution. GAs use operations such as crossover and mutation to generate new individuals from their "parent" individuals. This process relies on a competitive selection process that aims to identify good chromosome building blocks that occur frequencly in good candidate solutions.

A genetic algorithm works in the following way:

1) Initialize random population.

2) Calculate population fitness.

3) Select individuals for mating.

4) Mate selected individuals to produce new population. There is a random chance that individuals in the new population will be mutated.

5) Repeat from step 2) until an individual is fit enough or the maximum number of iterations is reached.


## Glossary

Before we continue, we will lay the basic terminology of the algorithm.

* Individual/Chromosome: A list of elements (called genes) that represent possible solutions.

* Population: The set of all the individuals.

* Fitness: An individual's score, calculated by a function specific to the problem.

Before we begin, let's mount Google Drive and add the folder to the search path.

This notebook requires the `search.py` and `utils.py` files used in the Week 2 & Week 3 practicals. Please ensure that you have placed it in the correct folder before running the notebook.

In [1]:
# mount Google Drive. You could also mount Google Drive using the Files tab on the left.
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ModuleNotFoundError:
    pass # we're not using Google Colab

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/')

## Activity 1: Map Colouring Problems

In this activity, you will implement the Australia Map Colouring Problem we have discussed in a previous week in the course. In this problem, we shall assign one of three colours to each region (state or territory) on the Australia Map such that adjacent regions are assigned different colours. We will ignore the Australian Capital Territory, and simply consider the following regions:

`WA`, `SA`, `NT`, `Q`, `NSW`, `VIC`, and `TAS`.

If you have not yet completed the Workbook exercises for this week, please examine them before attempting this practical.

First, we must define how to encode candidate solutions as chromosomes. Here, we will simply fix the order of the regions as described above and store the corresponding colour for each region in a list. For example, the candidate solution where WA is red, SA is blue, NT is green, Q is red, NSW is green, VIC is green, and TAS is red would be encoded as the list `[red, blue, green, red, green, green, red]`.

Now that we have defined a representation, it is time to define a fitness function. We will use a standard approach to fitness, where larger values of fitness indicate better individuals. For the map colouring problem, we can achieve this by counting how many adjacent territories have different colors. There are 9 shared boundaries, hence, the best fitness is 9 and the worst possible fitness of an individual is 0.

Define a function that computes the fitness of a given individual. The individual is represented as a list containing strings showing the colour assigned to each territory in the order defined above.

In [6]:
WA,SA,NT,Q,NSW,VIC,TAS = range(7)
ADJACENCIES = [(WA,SA),(WA,NT),(SA,NT),(SA,Q),(SA,NSW),(SA,VIC),(NT,Q),(Q,NSW),(NSW,VIC)]
COLORS=['red','green','blue']
def fitness_map(colors):
    # we count the violations (where adjacent regions have the same color)
    # and subtract that from the total number of adjacencies
    violations = sum(colors[i] == colors[j] for (i, j) in ADJACENCIES)
    return len(ADJACENCIES) - violations

To test our function, verify that it computes the fitness correctly for a few individuals:

In [7]:
for individual, correct_fitness in [(['red']*7, 0),
                                    (['red', 'red', 'green', 'red', 'green', 'green', 'red'], 6),
                                    (['red', 'blue', 'green', 'red', 'green', 'green', 'red'], 8),
                                    (['red', 'blue', 'green', 'red', 'green', 'red', 'blue'], 9)]:
    fitness = fitness_map(individual)
    print(f'Fitness for individual {individual} is {fitness}. That is {"CORRECT" if fitness==correct_fitness else "WRONG"}.')

Fitness for individual ['red', 'red', 'red', 'red', 'red', 'red', 'red'] is 0. That is CORRECT.
Fitness for individual ['red', 'red', 'green', 'red', 'green', 'green', 'red'] is 6. That is CORRECT.
Fitness for individual ['red', 'blue', 'green', 'red', 'green', 'green', 'red'] is 8. That is CORRECT.
Fitness for individual ['red', 'blue', 'green', 'red', 'green', 'red', 'blue'] is 9. That is CORRECT.


Next, we create an initial population that will be the starting point for the GA's search. We will do this by randomly selecting a colour for each region.

First, revisit how to create pseudo-random numbers in python. Read the documentation of functions `randrange`, `choice`, `choices`, and `random` defined in the `random` package at <https://docs.python.org/3/library/random.html>. We will use them throughout the remainder of this practical

Import the `random` package and try to generate a random numbers in the range [0,..,7].

Then, generate a list of length 7 which consists of randomly chosen colors.

In [10]:
import random
print(random.choices(COLORS, k=7))

0
['red', 'green', 'red', 'green', 'green', 'red', 'blue']


Now, define a function that generates a random initial population for the map colouring problem. Represent each chromosome as a list (as above), and represent the entire population as a list of lists.

In [17]:
def random_population(size, alphabet, chromosome_length=7):
    return [random.choices(alphabet, k=chromosome_length) for _ in range(size)]

Then, define the crossover operator. We will use the simple 1 point crossover in this practical. Define a function `crossover` that implements this operation. Return only one of the two possible offspring that can be generated by recombining two parent chromosomes.

In [23]:
def crossover(parent1, parent2):
    cutpoint = random.randrange(len(parent1)-1)+1
    offspring = parent1[:cutpoint] + parent2[cutpoint:]
    return offspring


Let's test that the function works correclty.

In [26]:
print(crossover([1,2,3,4,5],[6,7,8,9,10]))

[1, 2, 3, 9, 10]


Next, we define a mutation function that chooses a gene and assigns a value randomly chosen among a given set of candidate values. This function expects three arguments:

* the chromosome that will be mutated,
* the alphabet, that is, the possible values that can be assigned to a gene
* the probability of a mutation occurring. This floating point number between 0 and 1 defines how often the mutation is carried out. Higher values mean that it is more likley that a mutation will occur on a chromosome.

In [28]:
def mutate(chromosome, alphabet, probability):
    if (random.random() <= probability):
      mutation_point = random.randrange(len(chromosome))
      chromosome[mutation_point] = random.choice(alphabet)
    return chromosome



Let's test that the function works correctly.

In [31]:
mutate([1,2,3,4,5], COLORS, 0.5)

[1, 2, 'green', 4, 5]

Next, implement the overall GA algorithm. Define the function `ga_search` that takes as arguments

* the initial population,
* the fitness function,
* a fitness threshold which, when reached, terminates the search,
* crossover function,
* mutation function,
* number of iterations
* elitism flag, that, if true, applies elitism in the population. That is, if true, the individual with best fitness is carried over to the next population in each iteration.

The function shall return the iteration when the search stopped along with the best individual in the population at that iteration.

You may want to use the following sub-function that is defined in the `search` module:

* `select(n, population, fitness_fn)`: selects `n` individuals from `population` according to the Roulette Wheel Selection method using `fitness_fn` as the fitness function.

In [37]:
from search import select

def genetic_search(population, fitness_fn, fitness_min, crossover, mutate, iterations=100, verbose = False, elitism=False):
    fittest_individual = None
    for iteration in range(iterations):
        population = [mutate(crossover(*select(2, population, fitness_fn))) for _ in range(len(population))]
        if elitism and fittest_individual is not None:
            # replace the last individual in the population
            # alternatively, we could replace the one with lowest fitness.
            population[-1] = fittest_individual
        fittest_individual = max(population, key=fitness_fn)
        fittest_score = fitness_fn(fittest_individual)
        if verbose:
            print(f'Iteration: {iteration} best fitness: {fittest_score} individual: {fittest_individual}')
        if fittest_score >= fitness_min:
            return iteration, fittest_individual
    return iteration, max(population, key=fitness_fn)

Let's try to solve the map colouring problem with this algorihtm. Create an initial population of size 5, and create a mutation function that changes a region to a randomly selected color in 5% of all generated individuals. Invoke the `ga_search` function passing the initial population, fitness function, fitness score for terminating the search (9), crossover function, mutation function, maximum number of iterations (`1000`), and elitism (`True`). We pass score 9 as the desired termination criterion since that fitness score corresponds to a conflict-free solution. At this score, none of the 9 shared boundaries has a conflict.

In [48]:
initial_population_map = random_population(5, COLORS, chromosome_length=7)
mutate_map = lambda chromosome: mutate(chromosome, COLORS, 0.05)
iteration, solution = genetic_search(initial_population_map, fitness_map, 9, crossover, mutate_map, iterations=1000)
print(iteration, solution, fitness_map(solution))

999 ['green', 'red', 'red', 'red', 'green', 'blue', 'blue'] 6


Run the search a few times and see how different results are obtained. Sometimes, the GA finds an optimal solution with the desired score, while at other times, the search terminates without finding a solution that satisfies all conditions in the map colouring problem.

## Activity 2: GA for 8 Queens Problem

Now that you have a good grasp about how to implement GAs in python, attempt to solve the 8-queens problem using the GA function you defined earlier.

Recall that in the 8-Queens Problem we are tasked with placing 8 queens on an 8x8 chessboard so that no two queens can attack each other in a row, column, or diagonal.

Define a fitness function and a mutation function. Ensure that higher values of fitness coreespond to better candidate solutions, and ensure that the fitness value is always nonnegative.

Create an initial population of size 50. You may be able to reuse the function that generates the individual population.

Use mutation probability of 0.1, use elitism, and run the GA for at most 1000 iterations.

Since this problem is relatively difficult to solve with this simple GA, we may not always find a completely correct solution. Set the fitness threshold so that the search stops if there is at most one pair of queens that attack each other in a chromosome.

In [49]:
from itertools import combinations

def fitness_queens(queens):
    attacks = sum((qi[1]==qj[1]) or (abs(qi[0]-qj[0])==abs(qi[1]-qj[1])) for qi,qj in combinations(enumerate(queens), 2))
    return 28-attacks

In [57]:
ALPHABET_QUEENS = list(range(8))
population_queens = random_population(50, ALPHABET_QUEENS, chromosome_length=8)
mutate_queens = lambda chromosome: mutate(chromosome, ALPHABET_QUEENS, probability=0.1)

iteration, solution = genetic_search(population_queens, fitness_queens, 28, crossover, mutate_queens, iterations=1000, verbose=False, elitism=True)
print(iteration, solution, fitness_queens(solution))

999 [0, 2, 4, 6, 1, 3, 5, 0] 27


If you have time I encourage you to experiment a little with varying the size of the initial population, mutation probability, fitness threshold for stopping, number of iterations, and elitism. How does changing some of these parameters affect (a) the quality of the solution and (b) the runtime of the algorithm?

This concludes the practical. You should now be able to define appropriate chromosomes, mutation-, and crossover operations for a given problem, and find answers that are close to an optimal solution to the problem.
